# 第8章 数据规整：聚合、合并和重塑

In [2]:
import pandas as pd
import numpy as np

np.random.seed(2018)

## 8.1 层次化索引

层次化索引（hierarchical indexing）是pandas的⼀项重要功
能，它使你能在⼀个轴上拥有多个（两个以上）索引级别。抽象
点说，它使你能以低维度形式处理⾼维度数据。我们先来看⼀个
简单的例⼦：创建⼀个Series，并⽤⼀个由列表或数组组成的列
表作为索引：

In [8]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

可以看到的结果是经过美化的带有MultiIndex索引的Series的格式。
索引之间的“间隔”表示“直接使⽤上⾯的标签”：

In [10]:
data

a  1    0.433564
   2    0.510221
   3   -0.165131
b  1   -1.351779
   3    0.546631
c  1    1.230655
   2    1.076446
d  2   -1.210625
   3   -0.306677
dtype: float64

In [11]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

对于⼀个层次化索引的对象，可以使⽤所谓的部分索引，使⽤它
选取数据⼦集的操作更简单

In [5]:
data['b']


1   -1.279487
3    0.502277
dtype: float64

In [6]:
data['b':'c']

b  1   -1.279487
   3    0.502277
c  1    0.856029
   2   -0.142790
dtype: float64

In [13]:
data.loc[['b', 'c']]

b  1   -1.351779
   3    0.546631
c  1    1.230655
   2    1.076446
dtype: float64

有时候可以在“内层”进行选取

In [14]:
data.loc[:,2]

a    0.510221
c    1.076446
d   -1.210625
dtype: float64

层次化索引在数据重塑和基于分组的操作（如透视表⽣成）中扮
演着重要的⻆⾊。例如，可以通过unstack⽅法将这段数据重新
安排到⼀个DataFrame中：

In [15]:
data.unstack()

,1,2,3
a,0.433564,0.510221,-0.165131
b,-1.351779,NaN,0.546631
c,1.230655,1.076446,NaN
d,NaN,-1.210625,-0.306677


unstack的逆运算是stack:

In [16]:
data.unstack().stack()

a  1    0.433564
   2    0.510221
   3   -0.165131
b  1   -1.351779
   3    0.546631
c  1    1.230655
   2    1.076446
d  2   -1.210625
   3   -0.306677
dtype: float64

stack和unstack将在本章后面详细。对于一个DataFrame，每条轴都可以有分层索引

In [18]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

各层都可以有名字（可以是字符串，也可以是别的Python对
象）。如果指定了名称，它们就会显示在控制台输出中：

In [19]:
frame.index.names = ['key1', 'kwy2']

In [20]:
frame.columns.names = ['state', 'color']

In [21]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 kwy2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [23]:
frame['Ohio']

color      Green  Red
key1 kwy2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

可以单独创建MultiIndex然后复用。上面那个DataFrame中的（带有分级名称）列可以这样创建

In [24]:
MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], names=['state', 'color'])

NameError: name 'MultiIndex' is not defined